In [1]:
import numpy as np
import pandas as pd
from scipy.stats import laplace
import math
import statistics

data = np.array(pd.read_csv('dados/adult.csv'))

## Consultas

In [2]:
# Q1: Média do atributo age
def consulta1(data):
    idades = []
    for i in range(0, len(data)):
        idades.append(data[i][0])
    print("Média das idades: " + str(statistics.mean(idades)))
    return statistics.mean(idades)
    
# Q2: Somatório do atributo capital-gain
def consulta2(data):
    capital_gain = []
    for i in range(0, len(data)):
        capital_gain.append(data[i][10])
    print("Soma do capital: " + str(sum(capital_gain)))
    return sum(capital_gain)

# Q3: Média do atributo hours-per-week
def consulta3(data):
    horas = []
    for i in range(0, len(data)):
        horas.append(data[i][12])
    print("Média das horas: " + str(statistics.mean(horas)))
    return statistics.mean(horas)

# Q4: Quantidade de pessoas com salário maior que 50K
def consulta4(data):
    salario_alto = 0
    for i in range(0, len(data)):
        if data[i][14] == '>50K':
            salario_alto += 1
    print("Pessoas com salário alto: " + str(salario_alto))
    print()
    return salario_alto

## Resposta com ruído

In [3]:
def resposta_ruido(data, consulta, budget, sens):
    is_priv = False
    ruido = 0
    res = consulta(data)
    prob = laplace.pdf(0, loc=0, scale=sens / budget)
    while(not is_priv):
        ruido = laplace.rvs(loc=0, scale=sens / budget)
        res = res + ruido
        prob_ruido = laplace.pdf(ruido, loc=0, scale=sens / budget)
        if(prob <= ((math.e ** budget) * prob_ruido)):
            is_priv = True
            ruido = res
    return ruido

## Obtendo sensibilidades

In [4]:
def get_sense(consultas, data):
    query = []
    sense = []
    
    # adicionando as chamadas num array interno
    for c in consultas:
        query.append(c(data))
        
    # processando os atributos de um registro
    for reg in range(0, data.shape[0]):
        data_viz = np.copy(data)
        data_viz = np.delete(data_viz, reg, 0)
        for i in range(0, len(consultas)):
            res = consultas[i](data_viz)
            sens = abs(query[i] - res)
            sense.append(sens)
    return sense

## Gerando CSV

In [5]:
def gerar_csv(budgets, res, sens):
    resultado = []
    for i in range(0, len(budgets)):
        reg = [budgets[i]]
        
        for j in range(0, len(res[i])):
            reg.append(res[i][j])
            
        for j in range(0, len(sens)):
            reg.append(sens[j])
            
        resultado.append(reg)
    final_set = pd.DataFrame(datasetFinal, columns=['budget', 'result_q1', 'result_q2', 'result_q3', 'result_q4', 'sens_q1', 'sens_q2', 'sens_q3', 'sens_q4'])
    final_set.to_csv(r'saida/result.csv', index=False)

## Execução

In [6]:
budgets = [0.1, 1, 10]
consultas = [consulta1, consulta2, consulta3, consulta4]
sensibilidades = get_sense(consultas, data)

# Função principal
def dp_laplace(data, budgets, consultas, sens):
    println("Budgets: " + str(budgets))
    println("Consultas: " + str(consultas))
    println("Sensibilidades: " + str(sens))
    resultado = []
    for b in budgets:
        res_budget = []
        for i in range(0, len(consultas)):
            # adicionando a resposta com ruído às respostas de cada budget            res_budget.append(resposta_ruido(data, consultas[i], b / len(consultas), sens[i]))
        resultado.append(res_budget)
    gerar_csv(budget, resultado, sens)
    
dp_laplace(data, budgets, consultas, sensibilidades)